# Motivation to where it comes from

In [5]:
%%python2

class Old:
    pass

class New(object):
    pass

print 'old', type(Old)
print 'new', type(New)

old <type 'classobj'>
new <type 'type'>


In [11]:
%%python2

class New2:
    __metaclass__ = type

print 'new2:', type(New2)

new2: <type 'type'>


In [12]:
%%python2

__metaclass__ = type

class New3:
    pass

print 'new3', type(New3)

new3 <type 'type'>


# Building Classes Dynamically

In [14]:
class A(object):
    pass

In [16]:
A = type('A', (object,), {'x': 1}) # This is same as class A(object): pass

In [18]:
A.x

1

In [19]:
def __init__(self, x):
    self.x = x

def add(self, a, b):
    return a + b

In [20]:
C = type('C', (object, ), {'__init__': __init__, 'add': add})
# This is same as defining a class using the usual syntax

In [21]:
c = C(10)

In [22]:
c.x

10

In [25]:
c.add(4, 5)

9

# Lets try writing a metaclass and the only thing you have to do is inherit from type
if you inherit from object, you get a normal class, if you inherit from type (which is one level higher), then you get the new type

In [26]:
class NewMeta(type):
    def __str__(cls):
        # I am one level higher hence we recommend using cls
        # instead of self
        return 'Works'

In [31]:
# This is how it is done in Python 3
class B(metaclass=NewMeta):
    pass

In [32]:
str(B) # Should return 'Works'

'Works'

In [40]:
%%python2
# For python 2:

class NewMeta(type):
    def __str__(cls):
        # I am one level higher hence we recommend using cls
        # instead of self
        return 'Works'
    
class B:
    __metaclass__ = NewMeta

print B

Works


In [44]:
class MyMeta(type):
    def __new__(mcl, name, bases, cdict):
        """
        mcl - Metaclass Name
        name - Name of the class
        bases - Tuple of parent classes
        cdict - Dictionary of class Attributes
        """
        
        print('mcl:', mcl)
        print('name:', name)
        print('bases:', bases)
        print('cdict:', cdict)
        
        return super(MyMeta, mcl).__new__(mcl, name, bases, cdict)

In [45]:
class MyClass:
    __metaclass__ = MyMeta
    pass


mcl: <class '__main__.MyMeta'>
name: MyClass
bases: ()
cdict: {'__module__': '__main__', '__metaclass__': <class '__main__.MyMeta'>}


In [46]:
ls

Descriptors.ipynb                    descriptors_metaclasses_handout.pdf
Metaclasses.ipynb                    metaclasses/
deco_meta/                           properties/
deco_meta notes                      survey_link.txt
descriptors/


In [47]:
cd metaclasses/

/Users/deep/Desktop/deco_meta/metaclasses


In [48]:
ls

autometa_python2.py          prepare.py
base_conflict.py             problem_autometa_python2.py
class_deco.py                slotstyped.py*
classwatcher.py              submeta.py
meta_2_3.py                  use_autometa_python2.py
noclassattr.py*              use_classwatcher.py


In [ ]:
# %load meta_2_3.py
# file: meta_2_3.py

"""
The code is a bit hard to understand. The basic idea is exploiting the idea
that metaclasses can customize class creation and are picked by by the parent
class. This particular implementation uses a metaclass to remove its own parent
from the inheritance tree on subclassing. The end result is that the function
creates a dummy class with a dummy metaclass. Once subclassed the dummy
classes metaclass is used which has a constructor that basically instances a
new class from the original parent and the actually intended metaclass.
That way the dummy class and dummy metaclass never show up.

From:
 http://lucumr.pocoo.org/2013/5/21/porting-to-python-3-redux/#metaclass-syntax-changes

Used in:

* Jinja2
* SQLAlchemy
* future (python-future.org)

"""

from __future__ import print_function
import platform


# from jinja2/_compat.py
def with_metaclass(meta, *bases):
    # This requires a bit of explanation: the basic idea is to make a
    # dummy metaclass for one level of class instanciation that replaces
    # itself with the actual metaclass.  Because of internal type checks
    # we also need to make sure that we downgrade the custom metaclass
    # for one level to something closer to type (that's why __call__ and
    # __init__ comes back from type etc.).
    #
    # This has the advantage over six.with_metaclass in that it does not
    # introduce dummy classes into the final MRO.
    class metaclass(meta):
        __call__ = type.__call__
        __init__ = type.__init__

        def __new__(cls, name, this_bases, d):
            if this_bases is None:
                return type.__new__(cls, name, (), d)
            return meta(name, bases, d)

    return metaclass('temporary_class', None, {})


if __name__ == '__main__':

    class BaseClass(object):
        pass


    class MetaClass(type):
        """Metaclass for Python 2 and 3.
        """
        def __init__(cls, name, bases, cdict):
            print('It works with {impl} version {ver}.'.format(
                impl=platform.python_implementation(),
                ver=platform.python_version()))
            super(MetaClass, cls).__init__(name, bases, cdict)


    class Class(with_metaclass(MetaClass, BaseClass)):
        # BaseClass is optional.
        pass


In [56]:
# %load noclassattr.py
#file: noclassattr.py

"""Preventing non-callable class attributes with a metaclass.
"""

from __future__ import print_function

class NoClassAttributes(type):
    """No non-callable class attributes allowed
    """
    def __init__(cls, name, bases, cdict):
        allowed = set(['__module__', '__metaclass__', '__doc__',
                       '__qualname__'])
        for key, value in cdict.items():
            if (key not in allowed) and (not callable(value)):
                msg = 'Found non-callable class attribute "%s". ' % key
                msg += 'Only methods are allowed.'
                raise Exception(msg)
        super(NoClassAttributes, cls).__init__(name, bases, cdict)


if __name__ == '__main__':

    from meta_2_3 import with_metaclass

    class AttributeChecker:
        """Base class for meta.
        """
        __metaclass__ = NoClassAttributes
        pass

    class AttributeLess(AttributeChecker):
        """Only methods work.
        """
        def meth(self):
            """This is allowed'
            """
            print('Hello from AttributeLess.')

    attributeless = AttributeLess()
    attributeless.meth()


    class WithAttribute(AttributeChecker):
        """Has non-callable class attribute.
        Will raise an exception.
        """
        a = 10
        def meth(self):
            """This is allowed'
            """
            print('Hello from WithAttribute')



Hello from AttributeLess.


Exception: Found non-callable class attribute "a". Only methods are allowed.

In [62]:
# %load class_deco.py
# file: class_deco.py

def noclassattr_deco(cls):
    """Class decorator to allow only callable attributes.
    """
    allowed = set(['__module__', '__metaclass__', '__doc__', '__qualname__',
                   '__weakref__', '__dict__'])
    for key, value in cls.__dict__.items():
        if (key not in allowed) and (not callable(value)):
            msg = 'Found non-callable class attribute "%s". ' % key
            msg += 'Only methods are allowed.'
            raise Exception(msg)
    return cls


if __name__ == '__main__':

    @noclassattr_deco
    class AttributeLess(object):
        """Only methods work.
        """
        def meth(self):
            """This is allowed'
            """
            print('Hello from AttributeLess.')

    attributeless = AttributeLess()
    attributeless.meth()

    @noclassattr_deco
    class WithAttribute(object):
        """Has non-callable class attribute.
        Will raise an exception.
        """
        a = 10
        def meth(self):
            """This is allowed'
            """
            print('Hello from WithAttribute')

Hello from AttributeLess.


Exception: Found non-callable class attribute "a". Only methods are allowed.

In [64]:
# %load slotstyped.py
# file: slotstyped.py

"""Use of descriptor and metaclass to get slots with
given types.
"""

from __future__ import print_function

class TypDescriptor(object):
    """Descriptor with type.
    """

    def __init__(self, data_type, default_value=None):
        self.name = None
        self._internal_name = None
        self.data_type = data_type
        if default_value:
            self.default_value = default_value
        else:
            self.default_value = data_type()

    def __get__(self, instance, cls):
        return getattr(instance, self._internal_name, self.default_value)

    def __set__(self, instance, value):
        if not isinstance(value, self.data_type):
            raise TypeError('Required data type is %s. Got %s' % (
            self.data_type, type(value)))
        setattr(instance, self._internal_name, value)

    def __delete__(self, instance):
        raise AttributeError('Cannot delete %r' % instance)


class TypeProtected(type):
    """Metaclass to save descriptor values in slots.
    """

    def __new__(mcl, name, bases, cdict):
        slots = []
        for attr, value in cdict.items():
            if isinstance(value, TypDescriptor):
                value.name = attr
                value._internal_name = '_' + attr
                slots.append(value._internal_name)
        cdict['__slots__'] = slots
        return super(TypeProtected, mcl).__new__(mcl, name, bases, cdict)


if __name__ == '__main__':

    from meta_2_3 import with_metaclass


    class Typed(with_metaclass(TypeProtected)):
        pass

    class MyClass(Typed):
        """Test class."""
        attr1 = TypDescriptor(int)
        attr2 = TypDescriptor(float, 5.5)


    def main():
        """Test it.
        """
        my_inst = MyClass()
        print(my_inst.attr1)
        print(my_inst.attr2)
        print(dir(my_inst))
        print(my_inst.__slots__)
        my_inst.attr1 = 100
        print(my_inst.attr1)
        # this will fail
        try:
            my_inst.unknown = 100
        except AttributeError:
            print('cannot do this')

    main()


0
5.5
['__class__', '__delattr__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_attr1', '_attr2', 'attr1', 'attr2']
['_attr2', '_attr1']
100
cannot do this


In [66]:
# %load autometa_python2.py
# file: autometa_python2.py

"""Example usage of a metaclass.

We change the metaclass of classes that inherit from `object`.
"""

from __future__ import print_function

import __builtin__


class DebugMeta(type):
    """Metaclass to be used for debugging.

    """
    names = []
    counter = -1  # Do not count definition of new_object`.

    def __init__(cls, name, bases, cdict):
        """Store all class names and count how many classes are defined.
        """
        if DebugMeta.counter >= 0:
            DebugMeta.names.append('%s.%s' % (cls.__module__, name))
            super(DebugMeta, cls).__init__(name, bases, cdict)
        DebugMeta.counter += 1

    def report(cls):
        print('Defined %d classes.' % DebugMeta.counter)
        print(DebugMeta.names)


class new_object(object):
    """Replacement for the built-in `object`.
    """
    __metaclass__ = DebugMeta


def set_new_meta():
    """We actually change a built-in. This is a very strong measure.
    """
    __builtin__.object = new_object







In [68]:
# %load use_autometa_python2.py
# file: use_autometa_python2.py

from autometa_python2 import set_new_meta

set_new_meta()

class SomeClass1(object):
    """Test class.
    """
    pass


class SomeClass2(object):
    """Test class.
    """
    def __init__(self, arg1):
        self.arg1 = arg1

    def compute(self, arg2):
        return self.arg1 + arg2


class SomeClass3():
    """Test class. Does NOT inherit from object.
    """
    pass


if __name__ == '__main__':

    def test():
        """Make an instance and write the report.
        """
        inst = SomeClass2(10)
        assert inst.compute(10) == 20
        object.report()

    test()


Defined 2 classes.
['__main__.SomeClass1', '__main__.SomeClass2']


In [ ]:
# %load problem_autometa_python2.py
# file: problem_autometa_python2.py

from autometa_python2 import set_new_meta

set_new_meta()

import argparse
import doctest
import logging
import os
import shutil
import subprocess
import sys
import threading

# Comment out one of the lines below to see the problem.
# You may need to install the library.
# Try other large libraries you use.

# import matplotlib
# import numpy
# import sqlalchemy

object.report()
